In [6]:
%%writefile cuda_fft.cu
#include <cuda.h>
#include <cuComplex.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define PI 3.14159265358979323846

__device__ cuDoubleComplex complex_exp(double theta) {
    return make_cuDoubleComplex(cos(theta), sin(theta));
}

__global__ void fft_kernel(cuDoubleComplex *X, int N, int step) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int i = tid * step * 2;
    if (i + step < N) {
        for (int j = 0; j < step; j++) {
            cuDoubleComplex t = cuCmul(complex_exp(-2.0 * PI * j / (2.0 * step)), X[i + j + step]);
            cuDoubleComplex u = X[i + j];
            X[i + j] = cuCadd(u, t);
            X[i + j + step] = cuCsub(u, t);
        }
    }
}

__global__ void bit_reverse(cuDoubleComplex *X, int N, int logN) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= N) return;
    unsigned int rev = 0;
    unsigned int x = tid;
    for (int i = 0; i < logN; i++) {
        rev = (rev << 1) | (x & 1);
        x >>= 1;
    }
    if (rev > tid) {
        cuDoubleComplex temp = X[tid];
        X[tid] = X[rev];
        X[rev] = temp;
    }
}

void cuda_fft(cuDoubleComplex *h_X, int N) {
    cuDoubleComplex *d_X;
    cudaMalloc(&d_X, sizeof(cuDoubleComplex) * N);
    cudaMemcpy(d_X, h_X, sizeof(cuDoubleComplex) * N, cudaMemcpyHostToDevice);
    int logN = log2(N);
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    bit_reverse<<<numBlocks, blockSize>>>(d_X, N, logN);
    cudaDeviceSynchronize();
    for (int step = 1; step < N; step *= 2) {
        int numThreads = N / (2 * step);
        int blocks = (numThreads + blockSize - 1) / blockSize;
        fft_kernel<<<blocks, blockSize>>>(d_X, N, step);
        cudaDeviceSynchronize();
    }
    cudaMemcpy(h_X, d_X, sizeof(cuDoubleComplex) * N, cudaMemcpyDeviceToHost);
    cudaFree(d_X);
}

int next_power_of_2(int n) {
    int p = 1;
    while (p < n) p <<= 1;
    return p;
}

int main() {
    for (int n = 1; n <= 20; n++) {
        int N = next_power_of_2(n);
        cuDoubleComplex *x = (cuDoubleComplex *)malloc(N * sizeof(cuDoubleComplex));
        for (int i = 0; i < n; i++) {
            double real = rand() % 10;
            double imag = rand() % 10;
            x[i] = make_cuDoubleComplex(real, imag);
        }
        for (int i = n; i < N; i++) {
            x[i] = make_cuDoubleComplex(0, 0);
        }

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start);
        cuda_fft(x, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms = 0;
        cudaEventElapsedTime(&ms, start, stop);
        printf("n = %2d | Time taken: %.6f ms\n", n, ms);
        free(x);
    }
    return 0;
}


Overwriting cuda_fft.cu


In [7]:
!nvcc -o cuda_fft cuda_fft.cu

In [8]:
!./cuda_fft


n =  1 | Time taken: 0.794080 ms
n =  2 | Time taken: 0.348608 ms
n =  3 | Time taken: 0.330912 ms
n =  4 | Time taken: 0.335968 ms
n =  5 | Time taken: 0.332640 ms
n =  6 | Time taken: 0.322752 ms
n =  7 | Time taken: 0.335264 ms
n =  8 | Time taken: 0.352288 ms
n =  9 | Time taken: 0.423648 ms
n = 10 | Time taken: 0.370080 ms
n = 11 | Time taken: 0.394112 ms
n = 12 | Time taken: 0.362304 ms
n = 13 | Time taken: 0.371360 ms
n = 14 | Time taken: 0.370528 ms
n = 15 | Time taken: 0.366400 ms
n = 16 | Time taken: 0.375328 ms
n = 17 | Time taken: 0.435168 ms
n = 18 | Time taken: 0.464544 ms
n = 19 | Time taken: 0.432768 ms
n = 20 | Time taken: 0.444128 ms
